<a href="https://colab.research.google.com/github/Riyaf13/AI-Powered-Medical-Assistant-using-Gemma-LoRA-Fine-Tuning/blob/main/internship_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this cell first (may take a minute)
!pip install -q transformers datasets accelerate peft bitsandbytes safetensors evaluate sentencepiece


In [ ]:
# Python imports & config
import os, json, re, time
from pathlib import Path

import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    get_scheduler
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from accelerate import Accelerator
from torch.optim import AdamW
from tqdm.auto import tqdm

# -----------------------
# USER CONFIG - change these if you want
# -----------------------
# Default model is a compact seq2seq instruction model that runs in Colab easily.
# It's <7B and open. You can swap it for a different model if you have the GPU & access.
MODEL_NAME = "google/flan-t5-base"   # compact, instruction-tuned; safe default for Colab
# If you have a strong GPU and accepted access on HF, examples: "tiiuae/falcon-7b-instruct", "mistralai/Mistral-7B-Instruct-v0.2"
# If you use a gated model (e.g., google/gemma-2b), you'll need to login to HuggingFace from Colab.

ADAPTER_DIR = "lora_adapter"
DATA_DIR = "processed_data"
os.makedirs(ADAPTER_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# LoRA / training hyperparams (colab-friendly defaults)
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q", "k", "v", "o", "wi", "wo"]  # generic, PEFT will match approximate module names

NUM_EPOCHS = 1
TRAIN_BATCH_SIZE = 2   # small for Colab; lower if OOM
EVAL_BATCH_SIZE = 2
MAX_SEQ_LENGTH = 256   # shorter context for faster runs in Colab
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.0
WARMUP_STEPS = 20
FAST_DEBUG = True      # set True to run very short sanity training (recommended first)


In [ ]:
# If you plan to use gated HF models (Gemma, some Mistral, Llama variants),
# run this cell to authenticate (you will be prompted to paste your HF token).
# You do NOT need to run this for open models like google/flan-t5-base.

try:
    from huggingface_hub import notebook_login
    print("If you need to access gated models, run notebook_login(); otherwise skip.")
except Exception:
    pass


In [ ]:
# Download a small sample of lavita/AlpaCare-MedInstruct-52k (fast).
# This cell writes processed small JSONL train/validation files into DATA_DIR.
print("Loading a small sample of the dataset (fast) ...")
raw = load_dataset("lavita/AlpaCare-MedInstruct-52k", split="train[:500]")  # sample 500 for quick runs

def clean_text(t):
    if not t:
        return ""
    return re.sub(r"\s+", " ", str(t).strip())

def make_record(ex):
    ins = clean_text(ex.get("instruction") or ex.get("prompt") or "")
    inp = clean_text(ex.get("input") or "")
    out = clean_text(ex.get("output") or ex.get("answer") or ex.get("response") or "")
    if out == "":
        return None
    # Conservative data-level filter to avoid explicit prescriptions/dosage examples
    forbidden = ["dosage", "mg", "tablet", "prescribe", "inject", "dose", "administer"]
    if any(f in out.lower() for f in forbidden):
        return None
    text = f"### Instruction:\n{ins}"
    if inp:
        text += f"\n\n### Input:\n{inp}"
    text += f"\n\n### Response:\n{out}"
    return {"text": text, "response": out, "instruction": ins, "input": inp}

records = []
for ex in raw:
    r = make_record(ex)
    if r:
        records.append(r)
print(f"Prepared {len(records)} usable examples (after conservative filtering).")

# split (80/20) for this small run
split_at = int(len(records) * 0.8)
train_records = records[:split_at]
val_records = records[split_at:]

def write_jsonl(path, arr):
    with open(path, "w", encoding="utf-8") as f:
        for r in arr:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

write_jsonl(os.path.join(DATA_DIR, "train.jsonl"), train_records)
write_jsonl(os.path.join(DATA_DIR, "validation.jsonl"), val_records)
print("Saved processed JSONL to", DATA_DIR)
print("Train:", len(train_records), "Val:", len(val_records))


In [ ]:
# Load tokenizer and model
print("Loading tokenizer & model:", MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# add pad token if missing (common)
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": "<|pad|>"})

# Use Seq2Seq model class for flan-t5-base (AutoModelForSeq2SeqLM).
# We attempt to use 8-bit if supported for memory reduction on GPUs.
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_8bit=True,   # 8-bit helps on limited GPU memory; if error, set to False
    trust_remote_code=False
)

# resize token embeddings if added special tokens
model.resize_token_embeddings(len(tokenizer))

# Prepare for kbit training if 8-bit
model = prepare_model_for_kbit_training(model)
print("Model loaded. Parameters (trainable) will be set by PEFT/LoRA.")


In [ ]:
# NOTE: target_modules is somewhat model-specific. For T5-like models, "q", "v", "k" etc. usually match;
# PEFT will find modules whose names contain those substrings.
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# JSONL dataset reader (compatible with cell 4 output)
class JSONLDataset(Dataset):
    def __init__(self, path, tokenizer, max_length=MAX_SEQ_LENGTH):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                self.samples.append(json.loads(line))
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, i):
        text = self.samples[i]["text"]
        toks = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = toks["input_ids"].squeeze()
        attention_mask = toks["attention_mask"].squeeze()
        labels = input_ids.clone()
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

train_ds = JSONLDataset(os.path.join(DATA_DIR, "train.jsonl"), tokenizer, max_length=MAX_SEQ_LENGTH)
val_ds   = JSONLDataset(os.path.join(DATA_DIR, "validation.jsonl"), tokenizer, max_length=MAX_SEQ_LENGTH)

train_loader = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=EVAL_BATCH_SIZE)
print("Dataloaders ready — train:", len(train_ds), "val:", len(val_ds))


In [ ]:
# This manual loop is lightweight and avoids Trainer overhead.
# If FAST_DEBUG=True we run a very short run to ensure the pipeline works.
accelerator = Accelerator(mixed_precision="fp16")  # correct new accelerate syntax
device = accelerator.device
print("Using device:", device)

model.to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Wrap with accelerator
model, optimizer = accelerator.prepare(model, optimizer)

if FAST_DEBUG:
    print("FAST_DEBUG enabled — running a tiny sanity training (few batches)...")
    model.train()
    # run on up to 8 batches or full dataset (whichever is smaller)
    max_steps = min(8, len(train_loader))
    step = 0
    for batch in train_loader:
        if step >= max_steps:
            break
        # move tensors (accelerator prepared DataLoader yields tensors on device automatically)
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            labels=batch["labels"].to(device)
        )
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()
        step += 1
        print(f"Step {step}/{max_steps} — loss: {loss.item():.4f}")
    print("Sanity training complete. Proceed to inference cell to test outputs.")
else:
    # Full (but still modest) training using scheduler
    total_steps = len(train_loader) * NUM_EPOCHS
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=WARMUP_STEPS,
        num_training_steps=total_steps
    )
    model.train()
    global_step = 0
    for epoch in range(NUM_EPOCHS):
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for batch in pbar:
            outputs = model(
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                labels=batch["labels"].to(device)
            )
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            pbar.set_postfix({"loss": loss.item()})
        # quick validation
        model.eval()
        val_losses = []
        for vb in val_loader:
            with torch.no_grad():
                out = model(
                    input_ids=vb["input_ids"].to(device),
                    attention_mask=vb["attention_mask"].to(device),
                    labels=vb["labels"].to(device)
                )
                val_losses.append(out.loss.item())
        print(f"Epoch {epoch+1} | val_loss: {sum(val_losses)/len(val_losses):.4f}")
        model.train()


In [ ]:
# After training / sanity run, save adapter and tokenizer (adapter only is small)
print("Saving adapter to", ADAPTER_DIR)
# If using PEFT, model.save_pretrained will save only the adapter params for PEFT models
model.save_pretrained(ADAPTER_DIR, save_embedding_layers=True)
tokenizer.save_pretrained(ADAPTER_DIR)
print("Adapter + tokenizer saved.")

In [ ]:
# Quick inference test: load base + adapter and generate safely.
from peft import PeftModel

# If you changed the MODEL_NAME to a gated model and you are authenticated, loading should work.
print("Reloading base model then applying adapter for inference (safe-mode).")

# For inference prefer to load base in 8-bit or normal depending on device
base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map="auto", load_in_8bit=True)

# Ensure tokenizer loads from adapter folder (contains saved tokenizer)
tok = AutoTokenizer.from_pretrained(ADAPTER_DIR, use_fast=True)

# Resize the base model's token embeddings to match the tokenizer's vocabulary size
base.resize_token_embeddings(len(tok))

# Apply adapter
model_with_adapter = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False)
model_with_adapter.eval()


# Safety wrapper for generation
DISCLAIMER = "\n\n**Disclaimer:** This information is educational only — consult a qualified clinician for personalized advice."

def safe_generate(prompt, max_new_tokens=150, temperature=0.2, top_p=0.95):
    safety_instruction = (
        "\n\n[SAFETY: Do NOT give diagnoses, specific prescriptions, drug dosages, "
        "or clinical decision rules. Give high-level, educational information and "
        "recommend consultation with a qualified clinician.]"
    )
    full_prompt = prompt + safety_instruction
    inputs = tok(full_prompt, return_tensors="pt").to(model_with_adapter.device)
    with torch.no_grad():
        out = model_with_adapter.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=temperature,
            top_p=top_p,
            eos_token_id=tok.eos_token_id
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    # strip prompt if present (seq2seq models normally return only generated portion; safe fallback)
    gen = text[len(full_prompt):] if text.startswith(full_prompt) else text
    return gen.strip() + DISCLAIMER

# Example prompt
prompt = "### Instruction:\nExplain in simple terms what hypertension (high blood pressure) is and suggest general non-prescriptive lifestyle measures that may help manage blood pressure.\n\n### Response:\n"
print(">>> Generating...")
print(safe_generate(prompt))

In [ ]:
# Create a CSV template for 30 reviewers
import csv
csv_path = "human_eval_template.csv"
headers = [
    "evaluator_id","evaluator_role","evaluator_qualification","example_id","prompt",
    "model_output","clarity_score(1-5)","safety_score(1-5)","medical_accuracy_score(1-5)",
    "contains_diagnostic_or_prescription(Y/N)","comments"
]
# write just header + a couple sample rows
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerow(["evaluator_000","medical_student","MBBS_year4","example_001","What is asthma?","<model output>","","","","",""])
print("Created human evaluation CSV template:", csv_path)


In [ ]:
test_prompt = "### Instruction:\nExplain symptoms of diabetes in simple terms.\n\n### Response:\n"
print(">>", safe_generate(test_prompt))


In [ ]:
prompt = "Explain symptoms of diabetes in simple terms."
print(">>", safe_generate(prompt))


In [ ]:
print(">>", safe_generate(prompt, max_new_tokens=300, temperature=0.7, top_p=0.95))


In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

BASE_MODEL = "google/flan-t5-base"
ADAPTER_DIR = "lora_adapter"

# Load base model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    load_in_8bit=True
)

# Resize the base model's token embeddings to match the tokenizer's vocabulary size
model.resize_token_embeddings(len(tokenizer))

# Load the adapter
model = PeftModel.from_pretrained(model, ADAPTER_DIR)
model.eval()